In [76]:
import pandas as pd
import numpy as np
from statistics import mean, stdev
from matplotlib import pyplot as plt
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from datetime import datetime

In [77]:
startTime = datetime.now()

In [78]:
df = pd.read_csv('Datasets/TestCaseDataset.csv')

In [79]:
df.columns

Index(['Unnamed: 0', 'TEST ID', 'TEST CASE', 'PRE-CONDITIONS', 'PRECEDENCE',
       'COMPLEXITY', 'H', 'L', 'M', 'PRECON_COUNT', 'WEIGHTAGE'],
      dtype='object')

In [80]:
values = pd.DataFrame()
values['Id'] = range(0,1)
values['env'] = "Data"

In [81]:
x = df[['H','L','M','PRECON_COUNT','COMPLEXITY']]
y = df['WEIGHTAGE']

In [82]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [83]:
data_scaler = MinMaxScaler()
x_train = data_scaler.fit_transform(x_train)
x_test = data_scaler.transform(x_test)

In [84]:
def soft_acc(y_true, y_pred):
    return K.mean(K.equal(K.round(y_true), K.round(y_pred)))

In [85]:
model = Sequential()
model.add(Dense(10,input_shape=(5,),activation='relu'))
model.add(Dense(20,activation='relu'))
model.add(Dense(15,activation='relu'))
model.add(Dense(1,))

model.compile(Adam(lr=0.005),loss='mean_squared_error',metrics=[soft_acc])


C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [86]:
history = model.fit(x_train,y_train,epochs=10,validation_data = (x_test,y_test),validation_split=0.2,shuffle=True,verbose=0)

In [87]:
y_test_pred = model.predict(x_test)

1/1 [==============================] - 0s 341ms/step


In [88]:
Mse_list=[]
r2_list=[]
Mse_list.append(mean_squared_error(y_test,y_test_pred))
r2_list.append(r2_score(y_test,y_test_pred))

In [89]:
outcome = pd.DataFrame({'Actual':y_test,'Predicted':y_test_pred.flatten()})
outcome.head()

,Actual,Predicted
78,4.5,6.955570
35,23.4,7.349953
14,5.0,6.352601
69,4.5,6.955570
23,3.5,8.818038


In [90]:
values['Mse_list'] = mean(Mse_list)
mean(Mse_list)

65.24321132187684

In [91]:
values["R2_list"] = mean(r2_list)
mean(r2_list)

0.09151345139488987

In [92]:
    def save_figures(fig,filename):
        fig.savefig(os.path.join('plot/', filename + '.png'), bbox_inches='tight')

In [93]:
def loss_function(information):
    
    history_dict=information.history
    loss_values = history_dict['loss']
    val_loss_values=history_dict['val_loss']
    plt.plot(loss_values,'b--',label='training loss') # Training data loss
    plt.plot(val_loss_values,'r',label='training loss val') # Validation data loss
    plt.xlabel('Epochs',fontsize=22)
    plt.ylabel('Loss',fontsize=22)
    plt.title('Loss Curve',fontsize=22)
    save_figures(plt, 'loss_function')
    plt.close()


In [94]:
def actual_vs_prediction(y_test, y_test_pred):

    outcome = pd.DataFrame({'Actual': y_test,'Predicted': y_test_pred.flatten()})
    df_sorted = outcome.head(40).sort_values(by="Actual")

    df_sorted.plot(kind='bar', figsize=(12,7))
    plt.grid(which='major', linestyle='-', linewidth = '0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth = '0.5', color='black')
    plt.xlabel('Test Cases',fontsize=22)
    plt.ylabel('Priority Values',fontsize=22)
    plt.title("Comparision between 'Actual' and 'Predicted' values",fontsize=22)
    save_figures(plt, 'actual_vs_prediction')
    plt.close()
    
    plt.plot(df_sorted['Actual'].tolist(), label='Actual')
    plt.plot(df_sorted['Predicted'].tolist(), label='prediction')
    plt.xlabel('Test cases',fontsize=22)
    plt.ylabel('Priority Values',fontsize=22)
    plt.title("Comparision between 'Actual' and 'Predicted' values",fontsize=22)
    plt.grid(which='major', linestyle='-', linewidth = '0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth = '0.5', color='black')
    plt.legend()
    save_figures(plt, 'actual_vs_prediction_2')
    plt.close()

In [95]:
def regression_line(y_test, y_test_pred):
    
    fig, ax = plt.subplots()
    ax.scatter(y_test, y_test_pred)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
    ax.set_xlabel('Calculated by DeepOrder algorithm',fontsize=22)
    ax.set_ylabel('Predicted by Neural Network',fontsize=22)
    plt.title("Neural Network Regression Line",fontsize=22)
    plt.grid(which='major', linestyle='-', linewidth = '0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth = '0.5', color='black')
    save_figures(plt, 'regression_line')
    plt.close()

In [96]:
prep_time = datetime.now()-startTime

In [97]:
loss_function(history)
actual_vs_prediction(y_test,y_test_pred)
regression_line(y_test,y_test_pred)

In [98]:
start_prio_time = datetime.now()
s = data_scaler.fit_transform(df[['H','L','M','PRECON_COUNT','COMPLEXITY']])
A = model.predict(s)
fitness = A.ravel()

4/4 [==============================] - 0s 5ms/step


In [99]:
Dp_df = df.copy()
Dp_df

,Unnamed: 0,TEST ID,TEST CASE,PRE-CONDITIONS,PRECEDENCE,COMPLEXITY,H,L,M,PRECON_COUNT,WEIGHTAGE
0,0,LOAD_001,Test Page load functionality through URL,None,H,8,1,0,0,12,103.2
1,1,LOAD_002,Test Page Reload without crashes,None,H,3,1,0,0,0,2.7
2,2,LOAD_003,Test 404 Error for Invalid URL in domain,None,L,3,0,1,0,0,0.3
3,3,SELECT_CITY_001,Test City Choice using Icons,LOAD_001,M,6,0,0,1,1,9.0
4,4,SELECT_CITY_002,Test Search City using Search Bar,LOAD_001,M,5,0,0,1,1,7.5
...,...,...,...,...,...,...,...,...,...,...,...
95,95,BOOK_EVENT_003,Test Add & Remove person functionality,BOOK_EVENT_001,M,4,0,0,1,2,10.0
96,96,BOOK_EVENT_004,Test choice of seats,BOOK_EVENT_003,H,10,1,0,0,1,19.0
97,97,EVENT_CONFIRM_001,Test transfer to Payment Init,BOOK_EVENT_004,H,6,1,0,0,0,5.4
98,98,EVENT_CONFIRM_002,Test transfer to Confirmation Page after payment,PAY_PORTAL_003,H,7,1,0,0,0,6.3


In [100]:
Dp_df["Diff"] = (fitness - Dp_df['WEIGHTAGE'])

In [101]:
Dp_df.sort_values(by='Diff',axis=0,inplace=True)

In [102]:
Dp_df = Dp_df.drop(['H','L','M'],axis=True)

In [103]:
Dp_df

,Unnamed: 0,TEST ID,TEST CASE,PRE-CONDITIONS,PRECEDENCE,COMPLEXITY,PRECON_COUNT,WEIGHTAGE,Diff
42,42,PAY_INIT_001,Check load of Payment page,PAY_BUTTON_002,H,9,22,206.1,-197.256012
6,6,HOME_001,Test load of Home Page,LOAD_001,H,9,16,152.1,-143.509007
0,0,LOAD_001,Test Page load functionality through URL,None,H,8,12,103.2,-95.035766
73,73,PAY_PORTAL_003,Test redirect to Confirmation Page,PAY_PORTAL_002,H,8,8,71.2,-63.063275
72,72,PAY_PORTAL_002,Test Transaction confirmation,PAY_INIT_002,H,10,6,69.0,-60.090437
...,...,...,...,...,...,...,...,...,...
61,61,VOUCHER_002,Check Apply button,PAYMENT_OPT_003,M,2,0,1.0,5.461926
30,30,MOVIE_DETAILS_002,Check view of Rating of a Movie,MOVIE_LOAD_001,M,2,0,1.0,5.461926
29,29,MOVIE_DETAILS_001,Check view of Details of a Movie,MOVIE_LOAD_001,M,2,0,1.0,5.461926
80,80,CONFIRM_006,Check QR code display,CONFIRM_001,M,1,0,0.5,5.490704


In [104]:
Dp_df.to_csv("Results/DeepNN.csv")

In [105]:
from tensorflow.keras.models import load_model

model.save("Results/DeepNN.h5")